# check oof df

In [ ]:
import os
import gc
import yaml
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("/kaggle/src/dss_utils")
sys.path.append("/kaggle/src/exp")
sys.path.append("/kaggle/src/model")
sys.path.append("/kaggle/src/data")
from dss_metrics import score
from training_loop import get_valid_values_dict, concat_valid_input_info, get_oof_df, get_key_df
from dss_model import get_model
from dss_dataloader import get_loader


In [ ]:
output_dir = "/kaggle/working"
# exp_name = "exp014_td_event_fold0"
# exp_name = "exp016_inputtargetd3ch_epoch10"
exp_name = "exp016_inputtargetd3ch_bcewithlogits_epoch10_fold0"
# folds = [0, 1, 2, 3, 4]
folds = [0]
oof_df = pd.DataFrame()
for i in folds:
    print("fold", i)
    # df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"raw_oof_df_fold{i}.parquet"))
    df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"oof_df_fold{i}.parquet"))
    oof_df = pd.concat([oof_df, df], axis=0)


In [ ]:
oof_df = oof_df.sort_values(["series_id", "step"])
display(oof_df)


In [ ]:
# postprocess_fn
# series_idでgroupbyして、class_predに対して対象の列のデータから前のN個の列までのデータの平均をとる
import torch
import torch.nn as nn



def make_submission_from_eventdf(df, threshold=0.1, max_pool_size=3):
    df = df[df["second"] == 0].copy()
    df = df.sort_values(["series_id", "step"])
    df = df[["series_id", "step", "onset_pred", "wakeup_pred","minute"]].copy()
    df["step"] = df["step"].astype(np.float64)

    # onset_predとwakeup_predの大きい方をとる
    df["score"] = df[["onset_pred", "wakeup_pred"]].max(axis=1)
    # eventにonset_predが大きいときはonset、wakeup_predが大きいときはwakeupを入れる
    df = df.rename(columns={"onset_pred": "onset", "wakeup_pred": "wakeup"})
    df["event"] = df[["onset", "wakeup"]].idxmax(axis=1)
    df = df[df["score"] > threshold].reset_index(drop=True)
    df = df.sort_values(["series_id", "step"])
    return df


def happentime_process(df):
    freq_happen_time = np.array(
        [0, 3, 7, 11, 15, 18, 22, 26, 30, 33, 37, 41, 45, 48, 52, 56])
    # minuteがfreq_happen_timeのどれかに一番近いものを取得
    df["happen_time"] = df["minute"].apply(
        lambda x: freq_happen_time[np.argmin(np.abs(freq_happen_time - x))])
    df["diff_minute"] = df["minute"] - df["happen_time"]
    df["diff_step"] = df["diff_minute"] * 12
    # diff_stepずらしたものを増やす
    diff_df = df[df["diff_step"] != 0].copy()
    diff_df["step"] = diff_df["step"] + diff_df["diff_step"]
    diff_df["score"] = diff_df["score"] - 0.01
    df = df[df["diff_step"] == 0].copy()
    df["score"] = np.clip(df["score"] + 0.1, 0.0, 1.0)
    df = pd.concat([df, diff_df], axis=0)
    df = df.sort_values(["series_id", "step"])

    return df


In [ ]:
len(oof_df["series_id"].unique())


In [ ]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
train_event_df = train_event_df[train_event_df["step"].notnull()].copy()
train_event_df = train_event_df[train_event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
train_event_df = train_event_df.reset_index(drop=True)
display(train_event_df.head())


In [ ]:
# df = postprocess_notrolling(oof_df)
sub_df = make_submission_from_eventdf(df, threshold=0.1)
display(sub_df.head())


In [ ]:
oof_notroll_score = score(train_event_df, sub_df)
print(oof_notroll_score)
# happen_time_sub = happentime_process(sub_df)
# oof_notroll_score = score(train_event_df, happen_time_sub)
# print(oof_notroll_score)


In [ ]:
# average_size = 31
# max_kernel_size = 31
# df = postprocess_downsample_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
# sub_df = make_submission_df(df, threshold=0.1)
# ave_maxpool_score = score(train_event_df, sub_df)
# print(ave_maxpool_score)


In [ ]:
%%time
max_kernel_size = 3
tmp_score = 0
for average_size_ in range(3, 21, 2):
    print("average_size", average_size_, "max_kernel_size", max_kernel_size)
    df = postprocess_downsample_fn(oof_df, N=average_size_, maxpool_kernel_size=max_kernel_size)
    sub_df = make_submission_df(df, threshold=0.1)
    if len(sub_df) == 0:
        print("event not detected")
        continue
    score_ = score(train_event_df, sub_df)
    print(score_)
    if tmp_score > score_:
        break
    tmp_score = score_
    average_size = average_size_


In [ ]:
tmp_score = 0
max_kernel_size_ = max_kernel_size
for max_kernel_size in range(3, 31, 2):
    print("average_size", average_size, "max_kernel_size", max_kernel_size)
    df = postprocess_downsample_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
    sub_df = make_submission_df(df, threshold=0.1)
    if len(sub_df) == 0:
        print("event not detected")
        continue
    score_ = score(train_event_df, sub_df)
    print(score_)
    if tmp_score > score_:
        break
    tmp_score = score_
    max_kernel_size_ = max_kernel_size


In [ ]:
df = postprocess_downsample_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
sub_df = make_submission_df(df, threshold=0.01)
print(average_size, max_kernel_size)
print(len(sub_df))
if len(sub_df) == 0:
    print("event not detected")
    score_ = 0
else:
    score_ = score(train_event_df, sub_df)
print(score_)


"exp012_targetdownsample_epoch10"


average_size 9 max_kernel_size 3
0.7310038029952944

thr=0.01 -> 0.7469626949452008

---

exp_name = "exp012_targetdownsample_epoch5"

average_size 11 max_kernel_size 5
0.7325993414687024

0.7462916305317064